In [1]:
!pip install arch

Methods that test directly a type of RW

In [6]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import VarianceRatio
from scipy.stats import ttest_1samp

def get_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    data['Log_Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
    data.dropna(inplace=True)
    return data['Log_Returns']

# Variance Ratio Test pentru RW3
def variance_ratio_test(returns, lag=2):
    vr_test = VarianceRatio(returns, lags=lag)
    print("\nVariance Ratio Test:")
    print("Statistic:", vr_test.stat)
    print("P-value:", vr_test.pvalue)
    return vr_test.pvalue > 0.05  # True dacă nu respinge ipoteza RW3

# BDS test
def correlation_integral(time_series, m, epsilon):
    N = len(time_series)
    if N < m:
        raise ValueError(f"Lungimea seriei ({N}) trebuie să fie mai mare decât dimensiunea m ({m}).")

    # Creăm vectorii încorporați folosind o abordare de sliding window
    embedded_vectors = np.lib.stride_tricks.sliding_window_view(time_series, m)

    # Calculăm matricea distanțelor maxime între vectori
    distances = np.abs(embedded_vectors[:, None, :] - embedded_vectors[None, :, :]).max(axis=2)

    # Calculăm proporția perechilor de vectori cu distanța mai mică decât epsilon
    count = np.sum(distances < epsilon)
    total_pairs = distances.size
    return count / total_pairs
def bds_test(time_series, m=2, epsilon=0.01):
    N = len(time_series)

    # Calcularea corelației integrale pentru m și m-1
    C_m = correlation_integral(time_series, m, epsilon)
    C_m_minus_1 = correlation_integral(time_series, m - 1, epsilon)

    # Verificăm ca valorile corelației să nu fie zero
    if C_m <= 0 or C_m_minus_1 <= 0:
        raise ValueError("Corelația integrală este zero sau negativă, verificați parametrul epsilon sau seria temporală.")

    # Calculul statisticii BDS
    bds_stat = np.sqrt(N) * (np.log(C_m) - 0.5 * np.log(C_m_minus_1))
    p_value = 2 * (1 - norm.cdf(abs(bds_stat)))
    result = {
        "BDS Statistic": bds_stat,
        "P-value": p_value,
        "Reject H0": p_value < 0.05  # Respingem H0 dacă p-value < 0.05
    }
    return result

# Runs test
def runs_test(returns):
    signs = np.sign(returns)  # sign for each return
    n1 = np.sum(signs > 0)    # nr randamente pozitive
    n2 = np.sum(signs < 0)    # nr randamente negative
    n = n1 + n2               # total observații N=N(0)+N(1)

    # Nr secvente consecutive
    runs = np.sum(np.diff(signs) != 0) + 1

    # E[N] and Var[N]
    expected_runs = (2 * n1 * n2) / n + 1
    std_runs = np.sqrt((2 * n1 * n2 * (2 * n1 * n2 - n)) / (n**2 * (n - 1)))

    # Statistica Z
    z_stat = (runs - expected_runs) / std_runs

    print("\nRuns Test:")
    print("Total runs:", runs)
    print("E[N]:", expected_runs)
    print("Z-statistic:", z_stat)

    # True if we do not reject H0: pt is RW => RW1
    return abs(z_stat) < 1.96

# Testul Sequences and Reversals
def sequences_and_reversals_test(returns):
    sequences = np.sum(returns[:-1] * returns[1:] > 0)  # numărul de secvențe consecutive pozitive
    reversals = np.sum(returns[:-1] * returns[1:] < 0)  # numărul de inversări
    n = len(returns) - 1
    expected_sequences = n / 2
    expected_reversals = n / 2
    seq_z = (sequences - expected_sequences) / np.sqrt(n / 4)
    rev_z = (reversals - expected_reversals) / np.sqrt(n / 4)
    print("\nSequences and Reversals Test:")
    print("Z-statistic for sequences:", seq_z)
    print("Z-statistic for reversals:", rev_z)
    return (abs(seq_z) < 1.96) and (abs(rev_z) < 1.96)  # True dacă nu respinge ipoteza RW1

def test_random_walk(ticker, start, end):
    returns = get_data(ticker, start, end)
    print(f"Testing Random Walk for {ticker} from {start} to {end}")

    # RW3 - Variance Ratio Test
    rw3_result = variance_ratio_test(returns)
    if rw3_result:
        print("\nResult: RW3")
    else:
        print("\nResult: No RW3")

    # RW1 - BDS Test, Runs Test, Sequences and Reversals Test
    bds_result = bds_test(returns)
    runs_result = runs_test(returns)
    sequences_reversals_result = sequences_and_reversals_test(returns)

    if bds_result and runs_result and sequences_reversals_result:
        print("\nResult: RW1")
    else:
        print("\nResult: No RW1")

ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2024-01-01"
test_random_walk(ticker, start_date, end_date)


[*********************100%***********************]  1 of 1 completed

Testing Random Walk for AAPL from 2020-01-01 to 2024-01-01

Variance Ratio Test:
Statistic: -6.56622015984898
P-value: 5.1608495255095477e-11

Result: No RW3

Runs Test:
Total runs: 528
E[N]: 500.54491017964074
Z-statistic: 1.740606075422307

Sequences and Reversals Test:
Z-statistic for sequences: 31.433481274270598
Z-statistic for reversals: -31.68595903550972

Result: No RW1


Methods that test directly a type of RW + Methods that only test source necessary, but not sufficient condition

In [11]:
import yfinance as yf
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import acf
from arch.unitroot import VarianceRatio
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import bartlett, ttest_1samp

def get_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    data['Log_Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
    data.dropna(inplace=True)
    return data['Log_Returns']

# Variance Ratio Test for RW3
def variance_ratio_test(returns, lag=2):
    vr_test = VarianceRatio(returns, lags=lag)
    print("\nVariance Ratio Test:")
    print("Statistic:", vr_test.stat)
    print("P-value:", vr_test.pvalue)
    return vr_test.pvalue > 0.05  # True dacă nu respinge ipoteza RW3

# Sequences and Reversals Test
def sequences_and_reversals_test(returns):
    n = len(returns)
    m = np.mean(returns)
    s = np.std(returns)
    p = norm.cdf(m / s)

    # Calculăm numărul de secvențe și inversări
    sequences = np.sum(returns[:-1] * returns[1:] > 0)  # numărul de secvențe consecutive pozitive
    reversals = np.sum(returns[:-1] * returns[1:] <= 0)  # numărul de inversări

    # Statistica CJ (Cowles-Jones ratio)
    CJ = sequences / reversals  # CJ ratio

    expected_sequences = n / 2
    expected_reversals = n / 2
    seq_z = (sequences - expected_sequences) / np.sqrt(n / 4)
    rev_z = (reversals - expected_reversals) / np.sqrt(n / 4)
    cj_z = (CJ - 1) / np.sqrt((p * (1 - p)) / n)

    print("\nSequences and Reversals Test:")
    print("Z-statistic for sequences:", seq_z)
    print("Z-statistic for reversals:", rev_z)
    print("CJ ratio:", CJ)
    print("CJ Z-statistic:", cj_z)
    return (abs(seq_z) < 1.96) and (abs(rev_z) < 1.96) and (abs(cj_z) < 1.96)  # True dacă nu respinge ipoteza RW1


# Bartlett Test - pentru doua perioade
def bartlett_test(returns):
    n = len(returns)
    half = n // 2
    returns_first_half = returns[:half]
    returns_second_half = returns[half:]
    stat, p_value = bartlett(returns_first_half, returns_second_half)
    print("\n\nBartlett Test:")
    print("Statistic:", stat)
    print("P-value:", p_value)
    return p_value > 0.05  # True dacă nu respinge "uncorrelation"


# Testul Ljung-Box
def ljung_box_test(returns):
    max_lags = len(returns) - 1
    lb_result = acorr_ljungbox(returns, lags=list(range(1, max_lags+1)), return_df=True)
    print("\n\nLjung-Box Test:")
    lb_p_values = lb_result['lb_pvalue'].values
    reject_h0 = any(p_value < 0.05 for p_value in lb_p_values)  # respingem H0 dacă oricare p_value < 0.05
    return not reject_h0 # True if we do not reject H0: non-autocorrelation => Uncorrelation

# Box-Pierce test
def box_pierce_test(returns):
    max_lags = len(returns) - 1
    bp_result = acorr_ljungbox(returns, lags=list(range(1, max_lags+1)), return_df=True)

    print("\n\nBox-Pierce Test:")
    final_lag = max_lags
    bp_stat = bp_result['lb_stat'].values[-1]
    bp_p_value = bp_result['lb_pvalue'].values[-1]

    print("\nStatistic finală (pentru lagul {}):".format(final_lag))
    print("Statistic:", bp_stat)
    print("P-value:", bp_p_value)

    return bp_p_value > 0.05  # True if we do not reject H0: non-autocorrelation => Uncorrelation


# ARCH Effect Test
def arch_effect_test(returns):
    model = arch_model(returns, vol='ARCH', p=1)
    model_fitted = model.fit(disp="off")
    arch_test = model_fitted.arch_lm_test()
    arch_p_value = arch_test.pval
    print("\n\nARCH Effect Test:")
    print("P-value:", arch_p_value)
    return arch_p_value > 0.05  # True dacă nu respinge ipoteza de No Non-Liniar relation

# Ljung-Box for ARCH effects
def ljung_box_for_arch(returns, max_lags=10):
    # GARCH(1, 1)
    model = arch_model(returns, vol='Garch', p=1, q=1)
    model_fit = model.fit(disp="off")
    squared_residuals = model_fit.resid**2
    lb_result = acorr_ljungbox(squared_residuals, lags=list(range(1, max_lags + 1)), return_df=True)
    print("\n\nLjung-Box Test - Squared residulas for ARCH effects:")
    lb_p_values = lb_result['lb_pvalue'].values
    reject_h0 = any(p_value < 0.05 for p_value in lb_p_values)  # Respingem H0 dacă oricare p_value < 0.05
    return not reject_h0  # True dacă nu respingem H0: nu există autocorelare

def test_random_walk(ticker, start, end):
    returns = get_data(ticker, start, end)
    print(f"\nTesting Random Walk for {ticker} from {start} to {end}")

    # RW3 - Variance Ratio Test
    rw3_result = variance_ratio_test(returns)
    if rw3_result:
        print("\nResult: RW3")
    else:
        print("\nResult: No RW3")

    # RW1 - BDS Test, Runs Test, Sequences and Reversals Test
    bds_result = bds_test(returns, m=2, epsilon=0.01)
    runs_result = runs_test(returns)
    sequences_reversals_result = sequences_and_reversals_test(returns)

    print("\nBDS Test Results:")
    print(bds_result)

    if bds_result["Reject H0"] == False and runs_result and sequences_reversals_result:
        print("\nResult: RW1")
    else:
        print("\nResult: No RW1")

    # Uncorrelation Tests - Bartlett, Box Pierce, Ljung-Box, Variance Ratio Test
    bartlett_result = bartlett_test(returns)
    ljung_box_result = ljung_box_for_arch(returns)
    box_pierce_result = box_pierce_test(returns)
    variance_ratio_result = variance_ratio_test(returns)

    if bartlett_result and box_pierce_result and ljung_box_result and variance_ratio_result:
        print("\nResult: Uncorrelation")
    else:
        print("\nResult: No Uncorrelation")

    # Non-linear Relation Tests - ARCH Effect Test
    arch_result = arch_effect_test(returns)
    arch_ljung_box_result = ljung_box_for_arch(returns)

    if arch_result and arch_ljung_box_result:
        print("\nResult: No Non-Linear Relation")
    else:
        print("\nResult: Non-Linear Relation\n")

ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2024-01-01"
test_random_walk(ticker, start_date, end_date)


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004463. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004463. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/li


Testing Random Walk for AAPL from 2020-01-01 to 2024-01-01

Variance Ratio Test:
Statistic: -6.56622015984898
P-value: 5.1608495255095477e-11

Result: No RW3

Runs Test:
Total runs: 528
E[N]: 500.54491017964074
Z-statistic: 1.740606075422307

Sequences and Reversals Test:
Z-statistic for sequences: 31.38629481935646
Z-statistic for reversals: -31.512470878931765
CJ ratio: 333.3333333333333
CJ Z-statistic: 21085.023866236206

BDS Test Results:
{'BDS Statistic': -53.027045008409544, 'P-value': 0.0, 'Reject H0': True}

Result: No RW1


Bartlett Test:
Statistic: 32.93821093687798
P-value: 9.513489425031462e-09


Ljung-Box Test - Squared residulas for ARCH effects:


Box-Pierce Test:

Statistic finală (pentru lagul 1004):
Statistic: 1021.8753330460256
P-value: 0.34038206557235484

Variance Ratio Test:
Statistic: -6.56622015984898
P-value: 5.1608495255095477e-11

Result: No Uncorrelation


ARCH Effect Test:
P-value: 0.0


Ljung-Box Test - Squared residulas for ARCH effects:

Result: Non-Lin